In [139]:
# Dataset & DataLoader
# -Pytorch에서 배치크기만 데이터를 조절하기 위한 메거니즘
# -Dataset : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달
import numpy as np
import pandas as pd
import torch

x_data = torch.IntTensor([[10,20,30],[20,30,40],[30,40,50],[40,50,60],[50,60,70]])
y_data = torch.IntTensor([[20],[30],[40],[50],[60]])

# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset    
import torchvision

Dataset = TensorDataset(x_data, y_data) # 행 번호를 맞춰야 한다 

In [140]:
import pandas as pd
import numpy as np
irisDF=pd.read_csv("./iris.csv")
irisNP = np.loadtxt("./iris.csv",dtype=float, delimiter=',', usecols=[0, 1, 2, 3])

In [141]:
class DLDataset(TensorDataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data # 데이터프레임이라면 값들만, 아니라면 그냥 넘겨줌 
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        self.feature=torch.tensor(x_data)
        self.target=torch.tensor(y_data)

    # 데이터셋의 개수 체크 함수 콜백함수
    def __len__(self):
        return self.target.shape[0]

    # 특정 인덱스 데이터+라벨 반환 콜백함수
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [142]:
type(irisDF)  # 타입 확인하는 방법 

pandas.core.frame.DataFrame

In [143]:
if irisDF.__class__.__name__ == 'DataFrame': # 타입 확인하는 방법 
    print("DF")
else :
    print("----------")

DF


In [144]:
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.Series)

(True, False)

In [145]:
isinstance([10],list), isinstance({'A':22}, dict)

(True, True)

In [146]:
feature = irisDF[irisDF.columns[:-1]]
target = irisDF[irisDF.columns[-1]]
from sklearn.preprocessing import LabelEncoder
target = LabelEncoder().fit_transform(target)
target=target.reshape(-1,1)

my_dataset=DLDataset(feature, target) # 피쳐와 라벨로 분리 

In [147]:
my_dataset[0]

(tensor([4.9000, 3.0000, 1.4000, 0.2000], dtype=torch.float64), tensor([0]))

In [148]:
my_dataset2 = DLDataset(irisNP, target)

In [149]:
my_dataset2[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64), tensor([0]))

In [150]:
# 학습용, 검증용, 테스트용 Dataset
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
# 갯수로 주던가, 비율로 주던가 할 수 있다!
seed=torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset, [0.7,0.1,0.2], generator=seed)

print(f"trainDF => {len(trainDS)}, validDF => {len(validDS)}개, testDF => {len(testDS)}")
print(f"Subset 속성 => {trainDS.indices}") # 인덱스를 보여준다!
print(f"Subset 속성 => {trainDS.dataset}")

trainDF => 105, validDF => 15개, testDF => 29
Subset 속성 => [17, 100, 111, 49, 30, 124, 119, 55, 72, 142, 129, 93, 86, 103, 140, 13, 64, 108, 91, 25, 135, 73, 31, 82, 48, 106, 40, 44, 130, 24, 67, 3, 102, 54, 137, 63, 71, 61, 75, 59, 141, 107, 50, 112, 36, 8, 26, 11, 126, 29, 41, 74, 96, 128, 1, 33, 20, 105, 52, 42, 2, 39, 109, 62, 10, 28, 87, 121, 16, 65, 122, 104, 15, 69, 147, 84, 114, 58, 45, 110, 143, 144, 113, 27, 116, 6, 120, 18, 19, 32, 145, 134, 146, 98, 79, 9, 132, 5, 133, 23, 47, 66, 94, 136, 22]
Subset 속성 => <__main__.DLDataset object at 0x7faac67cba30>


In [151]:
batchsize=5
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False ]
trainDF=DataLoader(trainDS, batch_size=batchsize, shuffle=True)
validDF=DataLoader(validDS, batch_size=batchsize, shuffle=True)
testDF=DataLoader(testDS, batch_size=batchsize, shuffle=True)

# Epoch당 꺼내는 횟수 
len(trainDF), len(validDF), len(testDF) # 105/5=21, 15/5=3. 29/5=6

(21, 3, 6)

In [152]:
trainDS

In [153]:
for _, (feature,target) in enumerate(trainDF):
    print(f'[{_}] feature {feature}')

[0] feature tensor([[5.8000, 2.8000, 5.1000, 2.4000],
        [4.8000, 3.4000, 1.6000, 0.2000],
        [4.4000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.3000, 3.7000, 1.5000, 0.2000]], dtype=torch.float64)
[1] feature tensor([[6.5000, 3.0000, 5.2000, 2.0000],
        [6.1000, 2.6000, 5.6000, 1.4000],
        [5.4000, 3.4000, 1.7000, 0.2000],
        [5.9000, 3.2000, 4.8000, 1.8000],
        [5.4000, 3.4000, 1.5000, 0.4000]], dtype=torch.float64)
[2] feature tensor([[5.8000, 2.7000, 5.1000, 1.9000],
        [5.6000, 3.0000, 4.1000, 1.3000],
        [6.8000, 3.0000, 5.5000, 2.1000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [6.4000, 3.2000, 4.5000, 1.5000]], dtype=torch.float64)
[3] feature tensor([[6.0000, 2.2000, 4.0000, 1.0000],
        [5.8000, 2.6000, 4.0000, 1.2000],
        [7.4000, 2.8000, 6.1000, 1.9000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [4.9000, 2.5000, 4.5000, 1.7000]], dtype=torch.float64)
[4] feature tensor([[6.3

In [154]:
# 모델 생성
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class model(nn.Module) :
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(4, 3)

    def forward(self, x):
        x = self.fc1(x)
        return x
    

model=model()

In [155]:
# 실행 디바이스, 모델, 최적화, 손실함수, 학습횟수 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50
LOSS_FN=nn.CrossEntropyLoss().to(DEVICE)

In [166]:
optimizer=optim.Adam(model.parameters(), lr=0.001)

# 학습 진행함수
def training():
    model.train() # 학습모드로 설정 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화 

    for cnt, (feature, target) in enumerate(trainDF):
        # 모델 학습
        train_loss=[]
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        pre_target = model(feature.float())
        target = target.squeeze()
        


        loss=LOSS_FN(pre_target.argmax(dim=1).float(), target.float())
        train_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'[Train loss]==> {loss}')
    return train_loss

In [167]:
training()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
ex=torch.tensor([[-3.3297, -3.5757, -2.4487],
        [-1.2773, -2.1167, -1.1149],
        [-3.4242, -4.0081, -2.4938],
        [-1.9770, -2.5509, -1.3195],
        [-3.0342, -3.1301, -2.3086]])
ex.argmax(dim=1)

tensor([2, 2, 2, 2, 2])

In [ ]:
def testing():

    pass

In [ ]:
def predict():
    pass 

In [ ]:
for eps in range(EPOCHS):
    training()
    testing()

In [ ]:

criterion=nn.MSELoss()
for epoch in range(100):
    for i, (feature,target) in enumerate(trainDF):
        y_pre=model(feature.float())

        loss=criterion(y_pre, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()